## Tesla-Fabrik in Ostbrandenburg. Modell für Sentiment Analyse 
####In Notebook wird gezeigt , wie mit dem Modell auf Legestischer Regression man Sentiment Analyse durchführen kann
####Ziel ist über Sentiment Analyse verstehen, ob Einwohner in Berlin-Brandenburg echt für Tesla Gigafactory sind.

Ein gewisser Punkt der Arbeit ist Twitter API: 
API key
API secret key
Access token
Access token secret

In [25]:
appname <- "Tesla_Deutschland"
requestURL <- "https://api.twitter.com/oauth/request_token"
accessURL <- "http://api.twitter.com/oauth/request_token"
authURL <- "https://api.twitter.com/oauth/authorize"
consumerKey <- "w0PvfM0RYDOiSE0wt5cGwGvsE"
consumerSecret <- "86ICYBhyAXFJP0ptksNDC84IreCj5ra0wZzaAP9KLv8gnLTfny"
accessToken = "1257760441759039491-DAa9FuFn6i9E5rOGhck6LHMRRfmP9Z"
accessSecret = "mcRPGNHWWO5yd7oyxJoAppu2Yij5ZaiQyo9EbxmT1XCj8"

In [24]:
library("devtools")
library("usethis")
library("pradadata")
library("tidyverse")
library("twitteR")
library("tm")
library("wordcloud")

In [27]:
#Authorisierung
setup_twitter_oauth(consumer_key = consumerKey,
                    consumer_secret = consumerSecret,
                    access_token = accessToken,
                    access_secret = accessSecret)
#Das Suchen von Ergebnis. Max 10 blogs
tweets <- searchTwitter("#GigaBerlin" , lang ="de", n = 10)
tweets

[[1]]
[1] "JensWilhelm2: Berlin-Brandenburg, hier schon mal ein kleiner Vorgeschmack, was wohl passieren wird, wenn die Subventionen gefloss… https://t.co/Zg6fJiawIZ"

[[2]]
[1] "JensWilhelm2: \"Elon Musk wurde nicht wirklich ernst genommen\". Leider falsch, liebe @wiwo. Leider falsch. Elon Musk wurde viel zu… https://t.co/iFCUxhgvnI"

[[3]]
[1] "GermanTesla: RT @teslamag: Video: Erneut rege Bau-Vorbereitungen auf #Tesla-Grundstück für deutsche #Gigafactory https://t.co/xGIeJabhLb #Grünheide #Gig…"

In [28]:
#Die Daten sind nicht bearbeitet und müssen aufbereitet werden. Die Codierung ist "UTF8"
tweets%>%
  twListToDF() -> tweets_df
Encoding(tweets_df$text) <- "UTF8"

In [29]:
#Alle Daten werden in Corbus gespeichert für die weitere Filter 
corpus <- Corpus(VectorSource(tweets_df$text))
clearCorpus <- tm_map(corpus,function(x) iconv(enc2utf8(x), sub = "byte"))

In [30]:
tdm <- TermDocumentMatrix(clearCorpus, control = 
                            list(removePunctuation = TRUE,
                                 stopwords = c("com","https","@","\xe","?"),
                            removeNumbers = TRUE,
                            tolower = TRUE))


In [ ]:
<<TermDocumentMatrix (terms: 57, documents: 6)>>
Non-/sparse entries: 81/261
Sparsity           : 76%

In [31]:
#Die Daten (Wörter) können über Word Cloud visualisiert werden
m <- as.matrix(tdm)
word_freqs <- sort(rowSums(m),decreasing = TRUE)
dm <- data.frame(word=names(word_freqs),freq=word_freqs)
dm

                                                                     word freq
auf                                                                   auf    4
f\xe3\u0083\xe2�r                              f<e3><U+0083><e2><U+00BC>r    4
bauvorbereitungen                                       bauvorbereitungen    3
deutsche                                                         deutsche    3
erneut                                                             erneut    3
gigafactory                                                   gigafactory    3
rege                                                                 rege    3
teslagrundst\xe3\u0083\xe2�ck      teslagrundst<e3><U+0083><e2><U+00BC>ck    3
video                                                               video    3
elon                                                                 elon    2

In [23]:
wordcloud(dm$word, dm$freq,scale = c(3,.5),
          random.order = FALSE, colors = brewer.pal(8,"Dark2"))